In [1]:
import os
import ast
import numpy as np
import plotly
import plotly.graph_objects as go
import kaleido
import pandas as pd
import plotly.express as px
from sklearn.metrics import precision_recall_curve, average_precision_score
from statsmodels.nonparametric.smoothers_lowess import lowess

   ---------------------------------------- 0.0/9.8 MB ? eta -:--:--
   -- ------------------------------------- 0.5/9.8 MB 2.8 MB/s eta 0:00:04
   ----- ---------------------------------- 1.3/9.8 MB 3.9 MB/s eta 0:00:03
   -------- ------------------------------- 2.1/9.8 MB 3.6 MB/s eta 0:00:03
   ----------- ---------------------------- 2.9/9.8 MB 3.8 MB/s eta 0:00:02
   ------------------ --------------------- 4.5/9.8 MB 4.3 MB/s eta 0:00:02
   ------------------ --------------------- 4.5/9.8 MB 4.3 MB/s eta 0:00:02
   ------------------ --------------------- 4.5/9.8 MB 4.3 MB/s eta 0:00:02
   --------------------- ------------------ 5.2/9.8 MB 3.2 MB/s eta 0:00:02
   -------------------------- ------------- 6.6/9.8 MB 3.5 MB/s eta 0:00:01
   --------------------------------- ------ 8.1/9.8 MB 3.9 MB/s eta 0:00:01
   ---------------------------------------  9.7/9.8 MB 4.2 MB/s eta 0:00:01
   ---------------------------------------- 9.8/9.8 MB 4.2 MB/s eta 0:00:00
Note: you may need 


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Code for getting median, mean highest and lowest values for ball annotations in the test set

In [4]:
def process_txt_files(folder_path):
    """
    Processes text files in subfolders, extracting the last two positions from each list in the files,
    and computes the median, mean, highest, and lowest values for those positions across all files.

    :param folder_path: Path to the main folder containing subfolders with text files.
    :return: A dictionary with statistics for the last two positions.
    """
    position_3_values = []
    position_4_values = []

    for root, _, files in os.walk(folder_path):
        for file in files:
            if file.endswith('.txt'):
                file_path = os.path.join(root, file)
                with open(file_path, 'r') as f:
                    for line in f:
                        try:
                            # Safely evaluate the line to extract the list
                            data = ast.literal_eval(line.strip())
                            if isinstance(data, list) and len(data) >= 4:
                                position_3_values.append(data[2])  # 3rd position
                                position_4_values.append(data[3])  # 4th position
                        except (ValueError, SyntaxError):
                            # Skip lines that can't be parsed
                            continue

    # Compute statistics for position 3 and 4
    stats = {}
    for position, values in [(3, position_3_values), (4, position_4_values)]:
        if values:
            stats[f'Position {position}'] = {
                'Median': np.median(values),
                'Mean': np.mean(values),
                'Highest': max(values),
                'Lowest': min(values)
            }
        else:
            stats[f'Position {position}'] = 'No valid data'

    return stats



In [5]:
# Example usage:
folder_path = "C:\Studie\BEP\BEP_Football_tracking\Dataset_ball_eval\labelsfolders"
results = process_txt_files(folder_path)
print(results)

{'Position 3': {'Median': 17.0, 'Mean': 18.164100171099705, 'Highest': 46, 'Lowest': 3}, 'Position 4': {'Median': 16.0, 'Mean': 17.268315445636958, 'Highest': 48, 'Lowest': 3}}


In [52]:
def boxplot_widthheight(folder_path):
    """
    Processes text files in subfolders, extracting the last two positions from each list in the files,
    and computes the median, mean, highest, and lowest values for those positions across all files.

    :param folder_path: Path to the main folder containing subfolders with text files.
    :return: A dictionary with statistics for the last two positions and generates boxplots.
    """
    global fig
    position_3_values = []
    position_4_values = []

    for root, _, files in os.walk(folder_path):
        for file in files:
            if file.endswith('.txt'):
                file_path = os.path.join(root, file)
                with open(file_path, 'r') as f:
                    for line in f:
                        try:
                            # Safely evaluate the line to extract the list
                            data = ast.literal_eval(line.strip())
                            if isinstance(data, list) and len(data) >= 4:
                                position_3_values.append(data[2])  # 3rd position
                                position_4_values.append(data[3])  # 4th position
                        except (ValueError, SyntaxError):
                            # Skip lines that can't be parsed
                            continue

    # Compute statistics for position 3 and 4
    stats = {}
    for position, values in [(3, position_3_values), (4, position_4_values)]:
        if values:
            stats[f'Position {position}'] = {
                'Median': np.median(values),
                'Mean': np.mean(values),
                'Highest': max(values),
                'Lowest': min(values)
            }
        else:
            stats[f'Position {position}'] = 'No valid data'

    # Generate boxplots for position 3 and 4 using Plotly
    if position_3_values and position_4_values:
        fig = go.Figure()
        fig.add_trace(go.Box(y=position_3_values, name='Width', boxmean=False))
        fig.add_trace(go.Box(y=position_4_values, name='Height', boxmean=False))
        fig.update_layout(
            title='Dimension sizes of ball annotations in the test set in pixels',
            title_x=0.5,
            boxgap=0.3,
            yaxis_title='Size in pixels',
            boxmode='group',
            width=600,  # Set the desired width
            height=400
        )
        fig.show()

    return stats



In [53]:
# Example usage:
folder_path = "C:\Studie\BEP\BEP_Football_tracking\Dataset_ball_eval\labelsfolders"
results = boxplot_widthheight(folder_path)
print(results)

{'Position 3': {'Median': 17.0, 'Mean': 18.164100171099705, 'Highest': 46, 'Lowest': 3}, 'Position 4': {'Median': 16.0, 'Mean': 17.268315445636958, 'Highest': 48, 'Lowest': 3}}


In [2]:
y10traindfball = pd.read_csv('C:\Studie\BEP\BEP_Football_tracking\YOLO_results\Ball10/results.csv')
y8traindf = pd.read_csv('C:\Studie\BEP\BEP_Football_tracking\YOLO_results\players8/results.csv')
y9traindf = pd.read_csv('C:\Studie\BEP\BEP_Football_tracking\YOLO_results\players9/results.csv')
y10traindf = pd.read_csv('C:\Studie\BEP\BEP_Football_tracking\YOLO_results\players10/results.csv')
y11traindf = pd.read_csv('C:\Studie\BEP\BEP_Football_tracking\YOLO_results\players11/results.csv')

In [6]:
y10traindf.head()

,epoch,time,train/box_loss,train/cls_loss,train/dfl_loss,metrics/precision(B),metrics/recall(B),metrics/mAP50(B),metrics/mAP50-95(B),val/box_loss,val/cls_loss,val/dfl_loss,lr/pg0,lr/pg1,lr/pg2
0,1,1357.58,5.13263,13.57190,1.84534,0.48434,0.39750,0.31890,0.08650,5.63784,3.88997,2.02569,0.003332,0.003332,0.003332
1,2,2696.46,5.03556,3.46560,1.82583,0.51403,0.41356,0.36306,0.10418,5.60433,3.91540,2.01484,0.006644,0.006644,0.006644
2,3,4024.90,4.97554,3.40065,1.81797,0.53118,0.43505,0.38558,0.11719,5.26430,3.92990,1.96547,0.009933,0.009933,0.009933
3,4,5348.48,4.84774,3.24697,1.80274,0.54397,0.42350,0.39495,0.11673,5.51752,3.79030,1.99149,0.009901,0.009901,0.009901
4,5,6672.93,4.68485,3.03650,1.78567,0.52067,0.42320,0.38558,0.10721,5.57457,3.83061,1.99373,0.009868,0.009868,0.009868


In [14]:
# Example DataFrame
# Replace this with your actual dataframe
def vallossplot(df):
    # Create the figure
    fig = go.Figure()

    # Add a scatter plot with line
    fig.add_trace(go.Scatter(
        x=df['epoch'],
        y=df['metrics/mAP50-95(B)'],
        mode='lines+markers',
        name='Validation Box Loss'
    ))

    # Update layout
    fig.update_layout(

        title={
        'text': "MAP50-95 per epoch of training YOLO11 on persons",
        'x': 0.5,  # Center the title
        'xanchor': 'center',  # Anchor the title at the center
        'yanchor': 'top'  # Optional: Adjust vertical anchoring
        },
        xaxis_title="Epoch",
        yaxis_title="MAP50-95",
        yaxis=dict(range=[0, None]),
        template="plotly",
        width=700,  # Adjust the width of the plot
        height=500
    )

    # Show the plot
    fig.show()

In [7]:
vallossplot(y10traindfball)

In [9]:
vallossplot(y8traindf)

In [11]:
vallossplot(y9traindf)

In [13]:
vallossplot(y10traindf)

In [15]:
vallossplot(y11traindf)

In [11]:
data_hybrid = {
    "Sequence": [128, 129, 130, 131, 132, 133, 134, 135, 136, 137],
    "TP": [464, 445, 287, 383, 540, 556, 511, 475, 434, 604],
    "FP": [77, 86, 262, 73, 167, 150, 196, 53, 198, 74],
    "TN": [98, 181, 112, 180, 1, 1, 1, 3, 5, 0],
    "FN": [111, 38, 89, 114, 43, 43, 43, 219, 113, 72]
}

data_samurai = {
    "Sequence": [128, 129, 130, 131, 132, 133, 134, 135, 136, 137],
    "TP": [298, 5, 293, 297, 423, 603, 494, 256, 152, 531],
    "FP": [452, 745, 457, 453, 327, 147, 256, 494, 598, 219],
    "TN": [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    "FN": [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
}

# Convert to DataFrame
df_hybrid = pd.DataFrame(data_hybrid)
df_samurai = pd.DataFrame(data_samurai)

# Calculate F1 score
def calculate_f1_score(tp, fp, fn):
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    return f1

df_hybrid["F1"] = df_hybrid.apply(lambda row: calculate_f1_score(row["TP"], row["FP"], row["FN"]), axis=1)
df_samurai["F1"] = df_samurai.apply(lambda row: calculate_f1_score(row["TP"], row["FP"], row["FN"]), axis=1)

# Add a row for combined metrics
combined_hybrid_f1 = calculate_f1_score(df_hybrid["TP"].sum(), df_hybrid["FP"].sum(), df_hybrid["FN"].sum())
combined_samurai_f1 = calculate_f1_score(df_samurai["TP"].sum(), df_samurai["FP"].sum(), df_samurai["FN"].sum())

df_hybrid.loc["Combined"] = ["Combined"] + [None] * 4 + [combined_hybrid_f1]
df_samurai.loc["Combined"] = ["Combined"] + [None] * 4 + [combined_samurai_f1]

# Create the bar plot for F1 scores
fig = go.Figure()

fig.add_trace(go.Bar(
    x=df_hybrid["Sequence"].astype(str),
    y=df_hybrid["F1"],
    name="SAMURAI-YOLO hybrid",

))

fig.add_trace(go.Bar(
    x=df_samurai["Sequence"].astype(str),
    y=df_samurai["F1"],
    name="SAMURAI",
    marker_color="darkorange"

))

# Add "YOLO Ball" marker for combined row
yolo_f1_score = [None] * len(df_hybrid)  # Default values
yolo_f1_score[-1] = 0.73  # Set for the combined row
fig.add_trace(go.Bar(
    x=df_hybrid["Sequence"].astype(str),
    y=yolo_f1_score,
    name="YOLO-Ball",
    marker_color="green"
))

# Update layout
fig.update_layout(
    title={
        'text': "F1 score of SAMURAI and SAMURAI-YOLO hybrid on each sequence and total(including YOLO-ball)",
        'x': 0.5,  # Center the title
        'xanchor': 'center',  # Anchor the title at the center
        'yanchor': 'top'  # Optional: Adjust vertical anchoring
        },
    xaxis_title="Sequence number",
    yaxis_title="F1 Score",
    barmode="group",
    template="plotly",
    height=600,
    width=1000
)

# Show the plot
fig.show()

In [32]:
# Data from the table
confidence_thresholds = [0.001, 0.10, 0.20, 0.30, 0.40, 0.50, 0.60, 0.70, 0.80, 0.90]
f05_scores = [0.81, 0.85, 0.85, 0.85, 0.77, 0.72, 0.63, 0.63, 0.52, 0]
f1_scores = [0.70, 0.73, 0.73, 0.71, 0.66, 0.58, 0.49, 0.41, 0.30, 0.00]

# Create the plot
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=confidence_thresholds,
    y=f05_scores,
    mode='lines+markers',
    name='F0.5 Score',
    line=dict(color='blue', width=2),
    marker=dict(size=8)
))
fig.add_trace(go.Scatter(
    x=confidence_thresholds,
    y=f1_scores,
    mode='lines+markers',
    name='F1 Score',
    line=dict(color='darkorange', width=2),
    marker=dict(size=8)
))

# Customize layout
fig.update_layout(
    title={
        'text': "F Scores for different confidence thresholds Confidence Threshold",
        'x': 0.5,  # Center the title
        'xanchor': 'center',  # Anchor the title at the center
        'yanchor': 'top'  # Optional: Adjust vertical anchoring
        },
    xaxis=dict(title='Confidence Threshold', dtick=0.1),
    yaxis=dict(title='F Score', dtick=0.1, range=[0.0, 1]),
    template='plotly_white'
)

# Show the plot
fig.show()


In [39]:

# Provided data: [confidence_threshold, true_positive, false_positive, true_negative, false_negative]
confidence_data = [
    [0.001, 3765, 412, 578, 2745],
    [0.10, 3928, 265, 455, 2582],
    [0.20, 3826, 168, 822, 2684],
    [0.30, 3622, 99, 891, 2888],
    [0.40, 3215, 44, 946, 3295],
    [0.50, 2670, 24, 966, 3840],
    [0.60, 2132, 11, 979, 4378],
    [0.70, 1675, 4, 986, 4835],
    [0.80, 1142, 1, 989, 5368],
    [0.90, 0, 0, 990, 6510],
]

# Initialize lists for False Positive Rate and True Positive Rate
fpr = []
tpr = []

# Calculate FPR and TPR
for row in confidence_data:
    _, tp, fp, tn, fn = row
    # True Positive Rate (TPR) = TP / (TP + FN)
    tpr_value = tp / (tp + fn)
    # False Positive Rate (FPR) = FP / (FP + TN)
    fpr_value = fp / (fp + tn)

    tpr.append(tpr_value)
    fpr.append(fpr_value)

# Plot ROC curve using Plotly
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=fpr, y=tpr, mode='lines+markers',
    name='ROC Curve',
    line=dict(color='blue', width=2),
    marker=dict(color='darkorange', size=8)
))

# Plot diagonal line for reference (random classifier)
fig.add_trace(go.Scatter(
    x=[0, 1], y=[0, 1],
    mode='lines', name='Random Classifier',
    line=dict(color='gray', dash='dash')
))

fig.update_layout(
    title="Receiver Operating Characteristic (ROC) Curve",
    xaxis_title="False Positive Rate",
    yaxis_title="True Positive Rate",
    xaxis=dict(range=[0, 0.45]),
    yaxis=dict(range=[0, 1]),
    template = 'plotly',
    showlegend=True
)

# Show the plot
fig.show()

In [48]:
# Provided data: [confidence_threshold, true_positive, false_positive, true_negative, false_negative]
confidence_data = [
    #[0.001, 3765, 412, 578, 2745],
    [0.10, 3928, 265, 455, 2582],
    [0.20, 3826, 168, 822, 2684],
    [0.30, 3622, 99, 891, 2888],
    [0.40, 3215, 44, 946, 3295],
    [0.50, 2670, 24, 966, 3840],
    [0.60, 2132, 11, 979, 4378],
    [0.70, 1675, 4, 986, 4835],
    [0.80, 1142, 1, 989, 5368],
    [0.90, 0, 0, 990, 6510],
]

# Initialize lists for Precision and Recall
precision = []
recall = []

# Calculate Precision and Recall
for row in confidence_data:
    _, tp, fp, tn, fn = row
    # Precision = TP / (TP + FP)
    precision_value = tp / (tp + fp) if (tp + fp) != 0 else 0
    # Recall = TP / (TP + FN)
    recall_value = tp / (tp + fn) if (tp + fn) != 0 else 0

    precision.append(precision_value)
    recall.append(recall_value)

# Plot Precision-Recall Curve using Plotly
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=recall, y=precision, mode='lines+markers',
    name='Precision-Recall Curve',
    line=dict(color='blue', width=2),
    marker=dict(color='red', size=8)
))

# Update layout for better presentation
fig.update_layout(
    title="Precision-Recall Tradeoff Curve",
    xaxis_title="Recall (True Positive Rate)",
    yaxis_title="Precision",
    xaxis=dict(range=[0, 1]),
    yaxis=dict(range=[0, 1.1]),
    template = 'plotly',
    showlegend=True
)

# Show the plot
fig.show()

In [55]:
# Provided data: [confidence_threshold, true_positive, false_positive, true_negative, false_negative]
confidence_data = [
   # [0.001, 3765, 412, 578, 2745],
    [0.10, 3928, 265, 455, 2582],
    [0.20, 3826, 168, 822, 2684],
    [0.30, 3622, 99, 891, 2888],
    [0.40, 3215, 44, 946, 3295],
    [0.50, 2670, 24, 966, 3840],
    [0.60, 2132, 11, 979, 4378],
    [0.70, 1675, 4, 986, 4835],
    [0.80, 1142, 1, 989, 5368],
    [0.90, 0, 0, 990, 6510],
]

# Initialize lists for Precision and Recall
precision = []
recall = []

# Calculate Precision and Recall
for row in confidence_data:
    _, tp, fp, tn, fn = row
    # Precision = TP / (TP + FP)
    precision_value = tp / (tp + fp) if (tp + fp) != 0 else 0
    # Recall = TP / (TP + FN)
    recall_value = tp / (tp + fn) if (tp + fn) != 0 else 0

    precision.append(precision_value)
    recall.append(recall_value)

# Plot Precision-Recall Curve using Plotly with smoothing
fig = go.Figure()

# Plot the Precision-Recall curve with smoothing
fig.add_trace(go.Scatter(
    x=recall, y=precision, mode='lines+markers',
    name='Precision-Recall Curve',
    line=dict(color='blue', width=2, shape='spline'),
    marker=dict(color='red', size=8)
))

# Update layout for better presentation
fig.update_layout(
    title={
        'text': "F Scores for different confidence thresholds Confidence Threshold",
        'x': 0.5,  # Center the title
        'xanchor': 'center',  # Anchor the title at the center
        'yanchor': 'top'  # Optional: Adjust vertical anchoring
        },
    xaxis_title="Recall (True Positive Rate)",
    yaxis_title="Precision",
    xaxis=dict(range=[0, 1]),
    yaxis=dict(range=[0, 1.2]),
    template = 'plotly',
    showlegend=True
)

# Show the plot
fig.show()

In [58]:
# Provided data: [confidence_threshold, true_positive, false_positive, true_negative, false_negative]
confidence_data = [
    [0.001, 3765, 412, 578, 2745],
    [0.10, 3928, 265, 455, 2582],
    [0.20, 3826, 168, 822, 2684],
    [0.30, 3622, 99, 891, 2888],
    [0.40, 3215, 44, 946, 3295],
    [0.50, 2670, 24, 966, 3840],
    [0.60, 2132, 11, 979, 4378],
    [0.70, 1675, 4, 986, 4835],
    [0.80, 1142, 1, 989, 5368],
    [0.90, 0, 0, 990, 6510],
]

# Initialize lists for Precision and Recall
precision = []
recall = []

# Set maximum allowed precision value
max_precision = 1  # For example, clip the precision to not exceed 0.9

# Calculate Precision and Recall
for row in confidence_data:
    _, tp, fp, tn, fn = row
    # Precision = TP / (TP + FP)
    precision_value = tp / (tp + fp) if (tp + fp) != 0 else 0
    # Clip precision to the max value
    precision_value = min(precision_value, max_precision)
    # Recall = TP / (TP + FN)
    recall_value = tp / (tp + fn) if (tp + fn) != 0 else 0

    precision.append(precision_value)
    recall.append(recall_value)

# Plot Precision-Recall Curve using Plotly with smoothing
fig = go.Figure()

# Plot the Precision-Recall curve with smoothing
fig.add_trace(go.Scatter(
    x=recall, y=precision, mode='lines+markers',
    name='Precision-Recall Curve',
    line=dict(color='blue', width=2, shape='spline'),
    marker=dict(color='red', size=8)
))

# Update layout for better presentation
fig.update_layout(
    title="Smoothed Precision-Recall Tradeoff Curve (Clipped)",
    xaxis_title="Recall (True Positive Rate)",
    yaxis_title="Precision",
    xaxis=dict(range=[0, 1]),
    yaxis=dict(range=[0, 1.2]),
    showlegend=True
)

# Show the plot
fig.show()

In [2]:
data_samurai = {
    "Model": ['YOLO-Ball', 'SAMURAI', 'SAMURAI-YOLO Hybrid'],
    "TP": [3826, 3352, 4699],
    "FP": [168, 4148, 1336 ],
    "TN": [822,0,580],
    "FN": [2684, 0, 885],
    'P': [0.96, 0.45, 0.78],
    'R': [0.59, 1, 0.84],
    'F1': [0.73, 0.62, 0.81],
    'ACC': [0.62, 0.45, 0.70]
}

In [10]:
# Create DataFrame
df = pd.DataFrame(data_samurai)

# Normalize large-scale metrics as percentages of 7500
total = 7500
large_scale_cols = ['TP', 'FP', 'TN', 'FN']
df[large_scale_cols] = df[large_scale_cols].apply(lambda x: x / total)

# Combine metrics into one DataFrame for plotting
metrics = ['TP', 'FP', 'TN', 'FN', 'P', 'R', 'F1', 'ACC']
plot_data = pd.melt(df, id_vars=['Model'], value_vars=metrics,
                    var_name='Metric', value_name='Value')

# Create bar plot with Plotly
fig = go.Figure()

# Add traces for each model
models = df['Model'].unique()
colors = ['#2ca02c', '#ff7f0e', '#1f77b4']  # Custom colors for models
for i, model in enumerate(models):
    model_data = plot_data[plot_data['Model'] == model]
    fig.add_trace(go.Bar(
        x=model_data['Metric'],
        y=model_data['Value'],
        name=model,
        marker_color=colors[i]
    ))

# Update layout
fig.update_layout(
    title={
        'text': "Metrics for ball models with confusion matrix components scaled as a percentage",
        'x': 0.5,  # Center the title
        'xanchor': 'center',  # Anchor the title at the center
        'yanchor': 'top'  # Optional: Adjust vertical anchoring
        },
    xaxis_title="Metrics",
    yaxis_title="Metric Value",
    barmode='group',  # Group bars by Metric
    legend_title="Models",
    template='plotly'
)

# Show the plot
fig.show()

In [13]:
#Here put the results of YOLO into a df:
# Define the column names
columns = ["Threshold", "Class", "TP", "FP", "TN", "FN"]

# Create an empty DataFrame with these columns
df = pd.DataFrame(columns=columns)


# Add data manually (using .loc or append dictionaries)
df.loc[0] = [0.001, 'player', 91353, 5706, None,4029]  # First row
df.loc[1] = [0.001, 'referee', 8262, 719, None, 532]  # Second row
df.loc[2] = [0.001, 'goalkeepers', 3388, 519, None, 1349]  # Third ro
df.loc[3] = [0.001, 'background', None, 4730, None, 4531]  # Fourth row
df.loc[4] = [0.1, 'player', 92192,13242, None, 3190]
df.loc[5] = [0.1, 'referee', 8385, 2116, None, 409]
df.loc[6] = [0.1, 'goalkeepers', 3371, 1125, None, 1366]
df.loc[7] = [0.1, 'background', None, 3574, None, 15091]

df.loc[8] = [0.2, 'player', 91617, 7208, None, 3765]
df.loc[9] = [0.2, 'referee', 8319, 974, None, 475]
df.loc[10] = [0.2, 'goalkeepers', 3389, 658, None, 1348]
df.loc[11] = [0.2, 'background', None, 4365, None, 7617]

df.loc[12] = [0.3, 'player', 91037, 4682, None,4345]
df.loc[13] = [0.3, 'referee', 8190, 561, None, 604]
df.loc[14] = [0.3, 'goalkeepers', 3392, 403, None, 1345]
df.loc[15] = [0.3, 'background', None, 5175, None, 4531]

df.loc[16] = [0.4, 'player', 89122, 3041, None, 6260]
df.loc[17] = [0.4, 'referee', 7961, 337, None, 833]
df.loc[18] = [0.4, 'goalkeepers', 3315, 265, None, 1422]
df.loc[19] = [0.4, 'background', None, 1326, None, 2623]

df.loc[20] = [0.5, 'player', 85658, 1902, None, 9821]
df.loc[21] = [0.5, 'referee', 7565, 210, None, 1229]
df.loc[22] = [0.5, 'goalkeepers', 3173, 132, None, 4672]
df.loc[23] = [0.5, 'background', None, 11627, None, 1354]

df.loc[24] = [0.6, 'player', 80031, 1152, None, 15351]
df.loc[25] = [0.6, 'referee', 6916, 134, None, 1878]
df.loc[26] = [0.6, 'goalkeepers', 2921, 60, None, 1816]
df.loc[27] = [0.6, 'background', None, 18315, None, 616]

df.loc[28] = [0.7, 'player', 68487, 598, None, 26895]
df.loc[29] = [0.7, 'referee', 5786, 55, None, 3008]
df.loc[30] = [0.7, 'goalkeepers', 2365, 8, None, 2372]
df.loc[31] = [0.7, 'background', None, 31790, None, 176]

df.loc[32] = [0.8, 'player', 24005, 55, None, 71377]
df.loc[33] = [0.8, 'referee', 2944, 3, None, 5850]
df.loc[34] = [0.8, 'goalkeepers', 653, 0, None, 4084]
df.loc[35] = [0.8, 'background', None, 81282, None, 29]

df.loc[36] = [0.9, 'player', 1, 0, None, 95382]
df.loc[37] = [0.9, 'referee', 6, 0, None, 8788]
df.loc[38] = [0.9, 'goalkeepers', 1, 0, None, 4736]
df.loc[39] = [0.9, 'background', None, 108906, None, 0]
dfv11 = df

In [14]:


# Remove rows where Class is 'background'
colors = ['#2ca02c', '#ff7f0e', '#1f77b4']
df_filtered = df[df['Class'] != 'background']


df_filtered['Precision'] = df_filtered['TP'] / (df_filtered['TP'] + df_filtered['FP'])
df_filtered['Recall'] = df_filtered['TP'] / (df_filtered['TP'] + df_filtered['FN'])

df_sorted = df_filtered.sort_values(by=["Class", "Recall"])
# Plot using Plotly
fig = px.line(df_sorted, x="Recall", y="Precision", color="Class",
                 title="Precision vs Recall for different Classes",
                 labels={"Recall": "Recall", "Precision": "Precision", "Class": "Class"},
                 category_orders={"Class": ["player", "goalkeepers", "referee"]},
                 color_discrete_map={
                     "player": "#2ca02c",
                     "goalkeepers": "#ff7f0e",
                     "referee": "#1f77b4"
                 },
                 line_group="Class",


                 )


# Update layout
fig.update_traces(mode="lines+markers", line_shape="spline")
fig.update_layout(
    title={
        'text': "Precision-Recall curve per class for YOLO11",
        'x': 0.5,  # Center the title
        'xanchor': 'center',  # Anchor the title at the center
        'yanchor': 'top'  # Optional: Adjust vertical anchoring
        },
    xaxis_title="Recall",
    yaxis_title="Precision",
    legend_title="Class",
    template='plotly',
    xaxis=dict(range=[0, 1]),  # X-axis range
    yaxis=dict(range=[0.7, 1]) ,  # Y-axis range
    width=600,  # Set the width of the graph in pixels
    height=400  # Set the height of the graph in pixels
)



fig.show()

C:\Users\20212189\AppData\Local\Temp\ipykernel_14044\2160666143.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\20212189\AppData\Local\Temp\ipykernel_14044\2160666143.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [15]:
# Drop rows where class is 'background'
df_non_background = dfv11[dfv11['Class'] != 'background']

# Group by threshold
thresholds = df_non_background['Threshold'].unique()

print("Threshold\tTP\tFP\tFN\tPrecision\tRecall\tF1 Score")

for threshold in thresholds:
    subset = df_non_background[df_non_background['Threshold'] == threshold]

    # Compute totals for TP, FP, and FN
    total_tp = subset['TP'].sum()
    total_fp = subset['FP'].sum()
    total_fn = subset['FN'].sum()

    # Compute precision, recall, and F1 score
    precision = total_tp / (total_tp + total_fp) if (total_tp + total_fp) > 0 else 0
    recall = total_tp / (total_tp + total_fn) if (total_tp + total_fn) > 0 else 0
    f1_score = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    # Print results
    print(f"{threshold}\t{total_tp}\t{total_fp}\t{total_fn}\t{precision:.4f}\t{recall:.4f}\t{f1_score:.4f}")

Threshold	TP	FP	FN	Precision	Recall	F1 Score
0.001	103003	6944	5910	0.9368	0.9457	0.9413
0.1	103948	16483	4965	0.8631	0.9544	0.9065
0.2	103325	8840	5588	0.9212	0.9487	0.9347
0.3	102619	5646	6294	0.9479	0.9422	0.9450
0.4	100398	3643	8515	0.9650	0.9218	0.9429
0.5	96396	2244	15722	0.9773	0.8598	0.9148
0.6	89868	1346	19045	0.9852	0.8251	0.8981
0.7	76638	661	32275	0.9914	0.7037	0.8231
0.8	27602	58	81311	0.9979	0.2534	0.4042
0.9	8	0	108906	1.0000	0.0001	0.0001


In [16]:
# Drop rows where class is 'background'
df_non_background = dfv11[dfv11['Class'] != 'background']

# Group by threshold
thresholds = df_non_background['Threshold'].unique()

results = {}

for threshold in thresholds:
    subset = df_non_background[df_non_background['Threshold'] == threshold]

    # Compute totals for TP, FP, and FN
    total_tp = subset['TP'].sum()
    total_fp = subset['FP'].sum()
    total_fn = subset['FN'].sum()

    # Compute precision, recall, and F1 score
    precision = total_tp / (total_tp + total_fp) if (total_tp + total_fp) > 0 else 0
    recall = total_tp / (total_tp + total_fn) if (total_tp + total_fn) > 0 else 0
    f1_score = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    # Store results in a dictionary with the threshold as the key
    results[threshold] = [total_tp, total_fp, total_fn, precision, recall, f1_score]

print(results)

{0.001: [103003, 6944, 5910, 0.9368422967429761, 0.9457365052840341, 0.9412683907520789], 0.1: [103948, 16483, 4965, 0.8631332464232632, 0.9544131554543535, 0.9064810939026091], 0.2: [103325, 8840, 5588, 0.9211875362189631, 0.948692993490217, 0.9347379657858312], 0.3: [102619, 5646, 6294, 0.947850182422759, 0.942210755373555, 0.945022055640995], 0.4: [100398, 3643, 8515, 0.9649849578531541, 0.9218183320632064, 0.9429078580350686], 0.5: [96396, 2244, 15722, 0.9772506082725061, 0.8597727394352379, 0.9147553117793867], 0.6: [89868, 1346, 19045, 0.9852434933233933, 0.82513565873679, 0.898109700340284], 0.7: [76638, 661, 32275, 0.9914487897644213, 0.7036625563523179, 0.8231263291302388], 0.8: [27602, 58, 81311, 0.9979031091829357, 0.2534316380964623, 0.4042087381839749], 0.9: [8, 0, 108906, 1.0, 7.345244872100923e-05, 0.0001468941077101045]}


In [17]:
columns = ["Threshold", "Class", "TP", "FP", "TN", "FN"]

# Create an empty DataFrame with these columns
dfv10 = pd.DataFrame(columns=columns)

dfv10.loc[0] = [0.001, 'player', 91702, 9943, None,3680]  # First row
dfv10.loc[1] = [0.001, 'referee', 8285, 1103, None, 509]  # Se
dfv10.loc[2] = [0.001, 'goalkeepers', 3413, 368, None, 1324]
dfv10.loc[3] = [0.001, 'background', None, 4379, None, 4378]

dfv10.loc[4] = [0.1, 'player', 92789,21718, None, 2593]
dfv10.loc[5] = [0.1, 'referee', 8430, 2892, None, 364]
dfv10.loc[6] = [0.1, 'goalkeepers', 3396, 963, None, 1341]
dfv10.loc[7] = [0.1, 'background', None, 3027, None, 24302]

dfv10.loc[8] = [0.2, 'player', 92079, 13094, None, 3303]
dfv10.loc[9] = [0.2, 'referee', 8351, 1514, None, 443]
dfv10.loc[10] = [0.2, 'goalkeepers', 3414, 3909, None, 1323]
dfv10.loc[11] = [0.2, 'background', None, 3898, None, 13932]

dfv10.loc[12] = [0.3, 'player', 90991, 6886, None,4391]
dfv10.loc[13] = [0.3, 'referee', 8193, 704, None, 601]
dfv10.loc[14] = [0.3, 'goalkeepers', 3397, 264, None, 1340]
dfv10.loc[15] = [0.3, 'background', None, 5235, None, 6759]

dfv10.loc[16] = [0.4, 'player', 88311, 3453, None,7071]
dfv10.loc[17] = [0.4, 'referee', 7881, 249, None, 913]
dfv10.loc[18] = [0.4, 'goalkeepers', 3327, 115, None, 1410]
dfv10.loc[19] = [0.4, 'background', None, 8378, None, 2801]

dfv10.loc[20] = [0.5, 'player', 85418, 2199, None,9964]
dfv10.loc[21] = [0.5, 'referee', 7511, 146, None, 1283]
dfv10.loc[22] = [0.5, 'goalkeepers', 3161, 41, None, 1576]
dfv10.loc[23] = [0.5, 'background', None, 11902, None, 1465]

dfv10.loc[24] = [0.6, 'player', 82182, 1523, None,13200]
dfv10.loc[25] = [0.6, 'referee', 7095, 107, None, 1699]
dfv10.loc[26] = [0.6, 'goalkeepers', 2932, 27, None, 1805]
dfv10.loc[27] = [0.6, 'background', None, 15918, None, 871]

dfv10.loc[28] = [0.7, 'player', 77781, 1038, None,17597]
dfv10.loc[29] = [0.7, 'referee', 6591, 78, None, 2203]
dfv10.loc[30] = [0.7, 'goalkeepers', 2598, 15, None, 2139]
dfv10.loc[31] = [0.7, 'background', None, 21305, None, 493]

dfv10.loc[32] = [0.8, 'player', 67794, 538, None,27588]
dfv10.loc[33] = [0.8, 'referee', 5513, 50, None, 3282]
dfv10.loc[34] = [0.8, 'goalkeepers', 1862, 6, None, 2875]
dfv10.loc[35] = [0.8, 'background', None, 33367, None, 217]

dfv10.loc[36] = [0.9, 'player', 4511, 3, None,90871]
dfv10.loc[37] = [0.9, 'referee', 479, 0, None, 8315]
dfv10.loc[38] = [0.9, 'goalkeepers', 1, 0, None, 4737]
dfv10.loc[39] = [0.9, 'background', None, 103923, None, 3]

In [18]:


# Remove rows where Class is 'background'
colors = ['#2ca02c', '#ff7f0e', '#1f77b4']
df_filtered = dfv10[dfv10['Class'] != 'background']


df_filtered['Precision'] = df_filtered['TP'] / (df_filtered['TP'] + df_filtered['FP'])
df_filtered['Recall'] = df_filtered['TP'] / (df_filtered['TP'] + df_filtered['FN'])

df_sorted = df_filtered.sort_values(by=["Class", "Recall"])
# Plot using Plotly
fig = px.line(df_sorted, x="Recall", y="Precision", color="Class",
                 title="Precision vs Recall for different Classes",
                 labels={"Recall": "Recall", "Precision": "Precision", "Class": "Class"},
                 category_orders={"Class": ["player", "goalkeepers", "referee"]},
                 color_discrete_map={
                     "player": "#2ca02c",
                     "goalkeepers": "#ff7f0e",
                     "referee": "#1f77b4"
                 },
                 line_group="Class",


                 )


# Update layout
fig.update_traces(mode="lines+markers", line_shape="spline")
fig.update_layout(
    title={
        'text': "Precision-Recall curve per class for YOLOV10",
        'x': 0.5,  # Center the title
        'xanchor': 'center',  # Anchor the title at the center
        'yanchor': 'top'  # Optional: Adjust vertical anchoring
        },
    xaxis_title="Recall",
    yaxis_title="Precision",
    legend_title="Class",
    template='plotly',
    xaxis=dict(range=[0, 1]),  # X-axis range
    yaxis=dict(range=[0.4, 1]) ,  # Y-axis range
    width=600,  # Set the width of the graph in pixels
    height=400  # Set the height of the graph in pixels
)



fig.show()

C:\Users\20212189\AppData\Local\Temp\ipykernel_14044\577565930.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\20212189\AppData\Local\Temp\ipykernel_14044\577565930.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [19]:
# Drop rows where class is 'background'
df_non_background = dfv10[dfv10['Class'] != 'background']

# Group by threshold
thresholds = df_non_background['Threshold'].unique()

results = {}

for threshold in thresholds:
    subset = df_non_background[df_non_background['Threshold'] == threshold]

    # Compute totals for TP, FP, and FN
    total_tp = subset['TP'].sum()
    total_fp = subset['FP'].sum()
    total_fn = subset['FN'].sum()

    # Compute precision, recall, and F1 score
    precision = total_tp / (total_tp + total_fp) if (total_tp + total_fp) > 0 else 0
    recall = total_tp / (total_tp + total_fn) if (total_tp + total_fn) > 0 else 0
    f1_score = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    # Store results in a dictionary with the threshold as the key
    results[threshold] = [total_tp, total_fp, total_fn, precision, recall, f1_score]

print(results)

{0.001: [103400, 11414, 5513, 0.9005870364241295, 0.9493816165196074, 0.9243408260960904], 0.1: [104615, 25573, 4298, 0.8035686852858942, 0.9605373095957324, 0.8750695312859418], 0.2: [103844, 18517, 5069, 0.8486691020831801, 0.9534582648535987, 0.8980170706607749], 0.3: [102581, 7854, 6332, 0.9288812423597591, 0.9418618530386639, 0.9353265131207031], 0.4: [99519, 3817, 9394, 0.963062243555005, 0.9137476701587506, 0.9377570683489675], 0.5: [96090, 2386, 12823, 0.975770746171656, 0.88226382525502, 0.9266643843212512], 0.6: [92209, 1657, 16704, 0.9823471757611915, 0.8466298788941632, 0.9094531485015707], 0.7: [86970, 1131, 21939, 0.9871624612660469, 0.7985565931190259, 0.882899345210903], 0.8: [75169, 594, 33745, 0.9921597613610865, 0.6901683897386929, 0.8140591410949928], 0.9: [4991, 3, 103923, 0.999399279134962, 0.045825146445819635, 0.0876321241703831]}


In [20]:
columns = ["Threshold", "Class", "TP", "FP", "TN", "FN"]

# Create an empty DataFrame with these columns
dfv9 = pd.DataFrame(columns=columns)

dfv9.loc[0] = [0.001, 'player', 91530, 4888, None,3852]
dfv9.loc[1] = [0.001, 'referee', 8288, 637, None, 506]
dfv9.loc[2] = [0.001, 'goalkeepers', 3308, 492, None, 1429]
dfv9.loc[3] = [0.001, 'background', None, 4438, None, 4668]

dfv9.loc[4] = [0.1, 'player', 92300, 9522, None,3082]
dfv9.loc[5] = [0.1, 'referee', 8385, 1378, None, 409]
dfv9.loc[6] = [0.1, 'goalkeepers', 3289, 1091, None, 1448]
dfv9.loc[7] = [0.1, 'background', None, 3486, None, 10538]

dfv9.loc[8] = [0.2, 'player', 91828, 6054, None,3554]
dfv9.loc[9] = [0.2, 'referee', 8315, 816, None, 479]
dfv9.loc[10] = [0.2, 'goalkeepers', 3306, 631, None, 1431]
dfv9.loc[11] = [0.2, 'background', None, 4091, None, 6148]

dfv9.loc[12] = [0.3, 'player', 91252, 4079, None,4130]
dfv9.loc[13] = [0.3, 'referee', 8228, 482, None, 566]
dfv9.loc[14] = [0.3, 'goalkeepers', 3310, 372, None, 1427]
dfv9.loc[15] = [0.3, 'background', None, 4804, None, 3614]

dfv9.loc[16] = [0.4, 'player', 89754, 3005, None,5628]
dfv9.loc[17] = [0.4, 'referee', 8004, 300, None, 790]
dfv9.loc[18] = [0.4, 'goalkeepers', 3223, 207, None, 1514]
dfv9.loc[19] = [0.4, 'background', None, 6667, None, 2247]

dfv9.loc[20] = [0.5, 'player', 87006, 2150, None,8376]
dfv9.loc[21] = [0.5, 'referee', 7659, 203, None, 1135]
dfv9.loc[22] = [0.5, 'goalkeepers', 3096, 91, None, 1641]
dfv9.loc[23] = [0.5, 'background', None, 9997, None, 1289]

dfv9.loc[24] = [0.6, 'player', 82779, 1408, None,12603]
dfv9.loc[25] = [0.6, 'referee', 7172, 144, None, 1622]
dfv9.loc[26] = [0.6, 'goalkeepers', 2906, 26, None, 1831]
dfv9.loc[27] = [0.6, 'background', None, 15086, None, 608]

dfv9.loc[28] = [0.7, 'player', 74383, 914, None,20999]
dfv9.loc[29] = [0.7, 'referee', 6342, 78, None, 2452]
dfv9.loc[30] = [0.7, 'goalkeepers', 2518, 8, None, 2219]
dfv9.loc[31] = [0.7, 'background', None, 24957, None, 287]

dfv9.loc[32] = [0.8, 'player', 40100,321 , None,55282]
dfv9.loc[33] = [0.8, 'referee', 3210, 7, None, 5584]
dfv9.loc[34] = [0.8, 'goalkeepers', 1404, 0, None, 3333]
dfv9.loc[35] = [0.8, 'background', None, 63926, None, 55]

dfv9.loc[36] = [0.9, 'player', 1, 0, None,95382]
dfv9.loc[37] = [0.9, 'referee', 1, 0, None, 8794]
dfv9.loc[38] = [0.9, 'goalkeepers', 1, 0, None, 4737]
dfv9.loc[39] = [0.9, 'background', None, 108913, None, 0]

In [21]:


# Remove rows where Class is 'background'
colors = ['#2ca02c', '#ff7f0e', '#1f77b4']
df_filtered = dfv9[dfv9['Class'] != 'background']


df_filtered['Precision'] = df_filtered['TP'] / (df_filtered['TP'] + df_filtered['FP'])
df_filtered['Recall'] = df_filtered['TP'] / (df_filtered['TP'] + df_filtered['FN'])

df_sorted = df_filtered.sort_values(by=["Class", "Recall"])
# Plot using Plotly
fig = px.line(df_sorted, x="Recall", y="Precision", color="Class",
                 title="Precision vs Recall for different Classes",
                 labels={"Recall": "Recall", "Precision": "Precision", "Class": "Class"},
                 category_orders={"Class": ["player", "goalkeepers", "referee"]},
                 color_discrete_map={
                     "player": "#2ca02c",
                     "goalkeepers": "#ff7f0e",
                     "referee": "#1f77b4"
                 },
                 line_group="Class",


                 )


# Update layout
fig.update_traces(mode="lines+markers", line_shape="spline")
fig.update_layout(
    title={
        'text': "Precision-Recall curve per class for YOLOV9",
        'x': 0.5,  # Center the title
        'xanchor': 'center',  # Anchor the title at the center
        'yanchor': 'top'  # Optional: Adjust vertical anchoring
        },
    xaxis_title="Recall",
    yaxis_title="Precision",
    legend_title="Class",
    template='plotly',
    xaxis=dict(range=[0, 1]),  # X-axis range
    yaxis=dict(range=[0.7, 1]) ,  # Y-axis range
    width=600,  # Set the width of the graph in pixels
    height=400  # Set the height of the graph in pixels
)



fig.show()

C:\Users\20212189\AppData\Local\Temp\ipykernel_14044\1908619829.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\20212189\AppData\Local\Temp\ipykernel_14044\1908619829.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [22]:
# Drop rows where class is 'background'
df_non_background = dfv9[dfv9['Class'] != 'background']

# Group by threshold
thresholds = df_non_background['Threshold'].unique()

results = {}

for threshold in thresholds:
    subset = df_non_background[df_non_background['Threshold'] == threshold]

    # Compute totals for TP, FP, and FN
    total_tp = subset['TP'].sum()
    total_fp = subset['FP'].sum()
    total_fn = subset['FN'].sum()

    # Compute precision, recall, and F1 score
    precision = total_tp / (total_tp + total_fp) if (total_tp + total_fp) > 0 else 0
    recall = total_tp / (total_tp + total_fn) if (total_tp + total_fn) > 0 else 0
    f1_score = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    # Store results in a dictionary with the threshold as the key
    results[threshold] = [total_tp, total_fp, total_fn, precision, recall, f1_score]

print(results)

{0.001: [103126, 6017, 5787, 0.9448704910072108, 0.9468658470522343, 0.9458671167039658], 0.1: [103974, 11991, 4939, 0.8965981114991592, 0.9546518781045421, 0.924714734211439], 0.2: [103449, 7501, 5464, 0.932392969806219, 0.9498315168988092, 0.9410314605004025], 0.3: [102790, 4933, 6123, 0.9542066225411472, 0.9437808158805652, 0.9489650842888531], 0.4: [100981, 3512, 7932, 0.9663900931162853, 0.927171228411668, 0.946374516180426], 0.5: [97761, 2444, 11152, 0.9756099995010229, 0.8976063463498388, 0.9349840759762431], 0.6: [92857, 1578, 16056, 0.9832900937152539, 0.8525795818680966, 0.9132816649290871], 0.7: [83243, 1000, 25670, 0.9881295775316643, 0.7643072911406352, 0.8619250761042888], 0.8: [44714, 328, 64199, 0.9927179077305626, 0.41054786848218305, 0.5808710337436264], 0.9: [3, 0, 108913, 1.0, 2.7544162473833045e-05, 5.508680762768663e-05]}


In [23]:
columns = ["Threshold", "Class", "TP", "FP", "TN", "FN"]

# Create an empty DataFrame with these columns
dfv8 = pd.DataFrame(columns=columns)

dfv8.loc[0] = [0.001, 'player', 91559, 5163, None,3823]
dfv8.loc[1] = [0.001, 'referee', 8296, 416, None, 498]
dfv8.loc[2] = [0.001, 'goalkeepers', 3004, 530, None, 1733]
dfv8.loc[3] = [0.001, 'background', None, 4441, None, 4496]

dfv8.loc[4] = [0.1, 'player', 92432, 11835, None,2950]
dfv8.loc[5] = [0.1, 'referee', 8403, 1287, None, 391]
dfv8.loc[6] = [0.1, 'goalkeepers', 2976, 1185, None, 1761]
dfv8.loc[7] = [0.1, 'background', None, 3351, None, 12556]

dfv8.loc[8] = [0.2, 'player', 91843, 6562, None,3539]
dfv8.loc[9] = [0.2, 'referee', 8343, 579, None, 451]
dfv8.loc[10] = [0.2, 'goalkeepers', 3010, 674, None, 1727]
dfv8.loc[11] = [0.2, 'background', None, 4085, None, 6183]

dfv8.loc[12] = [0.3, 'player', 91324, 4478, None,4058]
dfv8.loc[13] = [0.3, 'referee', 8244, 324, None, 550]
dfv8.loc[14] = [0.3, 'goalkeepers', 3003, 413, None, 1734]
dfv8.loc[15] = [0.3, 'background', None, 4756, None, 3629]

dfv8.loc[16] = [0.4, 'player', 89700, 3229, None,5682]
dfv8.loc[17] = [0.4, 'referee', 8059, 193, None, 735]
dfv8.loc[18] = [0.4, 'goalkeepers', 2960, 220, None, 1777]
dfv8.loc[19] = [0.4, 'background', None, 6652, None, 2100]

dfv8.loc[20] = [0.5, 'player', 86658, 2314, None,8724]
dfv8.loc[21] = [0.5, 'referee', 7726, 112, None, 1068]
dfv8.loc[22] = [0.5, 'goalkeepers', 2823, 77, None, 1914]
dfv8.loc[23] = [0.5, 'background', None, 10268, None, 1065]

dfv8.loc[24] = [0.6, 'player', 81594, 1669, None,13788]
dfv8.loc[25] = [0.6, 'referee', 7114,73, None, 1680]
dfv8.loc[26] = [0.6, 'goalkeepers', 2556, 15, None, 2181]
dfv8.loc[27] = [0.6, 'background', None, 16404, None, 512]

dfv8.loc[28] = [0.7, 'player', 71172, 1082, None,24210]
dfv8.loc[29] = [0.7, 'referee', 6032, 44, None, 2762]
dfv8.loc[30] = [0.7, 'goalkeepers', 2027, 3, None, 2710]
dfv8.loc[31] = [0.7, 'background', None, 28775, None, 222]

dfv8.loc[32] = [0.8, 'player', 32159, 309, None,63223]
dfv8.loc[33] = [0.8, 'referee', 2704, 3, None, 6090]
dfv8.loc[34] = [0.8, 'goalkeepers', 943, 0, None, 3794]
dfv8.loc[35] = [0.8, 'background', None, 72837, None, 42]

dfv8.loc[36] = [0.9, 'player', 5, 0, None,95377]
dfv8.loc[37] = [0.9, 'referee', 3, 0, None, 8791]
dfv8.loc[38] = [0.9, 'goalkeepers', 1, 0, None, 4737]
dfv8.loc[39] = [0.9, 'background', None, 108905, None, 0]

In [24]:
#make a plot of F1 score per class over confidence interval for each model

In [25]:


# Remove rows where Class is 'background'
colors = ['#2ca02c', '#ff7f0e', '#1f77b4']
df_filtered = dfv8[dfv8['Class'] != 'background']


df_filtered['Precision'] = df_filtered['TP'] / (df_filtered['TP'] + df_filtered['FP'])
df_filtered['Recall'] = df_filtered['TP'] / (df_filtered['TP'] + df_filtered['FN'])

df_sorted = df_filtered.sort_values(by=["Class", "Recall"])
# Plot using Plotly
fig = px.line(df_sorted, x="Recall", y="Precision", color="Class",
                 title="Precision vs Recall for different Classes",
                 labels={"Recall": "Recall", "Precision": "Precision", "Class": "Class"},
                 category_orders={"Class": ["player", "goalkeepers", "referee"]},
                 color_discrete_map={
                     "player": "#2ca02c",
                     "goalkeepers": "#ff7f0e",
                     "referee": "#1f77b4"
                 },
                 line_group="Class",


                 )


# Update layout
fig.update_traces(mode="lines+markers", line_shape="spline")
fig.update_layout(
    title={
        'text': "Precision-Recall curve per class for YOLOV8",
        'x': 0.5,  # Center the title
        'xanchor': 'center',  # Anchor the title at the center
        'yanchor': 'top'  # Optional: Adjust vertical anchoring
        },
    xaxis_title="Recall",
    yaxis_title="Precision",
    legend_title="Class",
    template='plotly',
    xaxis=dict(range=[0, 1]),  # X-axis range
    yaxis=dict(range=[0.7, 1]) ,  # Y-axis range
    width=600,  # Set the width of the graph in pixels
    height=400  # Set the height of the graph in pixels
)



fig.show()

C:\Users\20212189\AppData\Local\Temp\ipykernel_14044\640014700.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\20212189\AppData\Local\Temp\ipykernel_14044\640014700.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [32]:
# Assuming dfv8, dfv9, dfv10, and df are already defined pandas dataframes
def compute_f1_scores(df):
    # Filter rows where 'Class' == 'player'
    df_player = df[df['Class'] == 'player']

    # Compute precision, recall, and F1 score
    df_player['Precision'] = df_player['TP'] / (df_player['TP'] + df_player['FP'])
    df_player['Recall'] = df_player['TP'] / (df_player['TP'] + df_player['FN'])
    df_player['F1'] = 2 * (df_player['Precision'] * df_player['Recall']) / (df_player['Precision'] + df_player['Recall'])

    # Group by 'Threshold' and compute the mean F1 score for each threshold
    return df_player.groupby('Threshold')['F1'].mean()

# Compute F1 scores for each dataframe
f1_v8 = compute_f1_scores(dfv8)
f1_v9 = compute_f1_scores(dfv9)
f1_v10 = compute_f1_scores(dfv10)
f1 = compute_f1_scores(df)

# Create the line plot
fig = go.Figure()
colors = ['#2ca02c', '#ff7f0e', '#1f77b4', '#d62728']
# Add traces for each dataframe
fig.add_trace(go.Scatter(x=f1_v8.index, y=f1_v8, mode='lines+markers', name='YOLOV8', line=dict(color=colors[0])))
fig.add_trace(go.Scatter(x=f1_v9.index, y=f1_v9, mode='lines+markers', name='YOLOV9', line=dict(color=colors[1])))
fig.add_trace(go.Scatter(x=f1_v10.index, y=f1_v10, mode='lines+markers', name='YOLOV10', line=dict(color=colors[2])))
fig.add_trace(go.Scatter(x=f1.index, y=f1, mode='lines+markers', name='YOLO11', line=dict(color=colors[3])))

# Customize the layout
fig.update_layout(
    title={'text': 'F1-score for player class at various confidence thresholds', 'x': 0.5, 'xanchor': 'center'},
    xaxis_title='Confidence Threshold',
    yaxis_title='F1-Score',
    template='plotly',
    width=800,  # Set the width of the graph in pixels
    height=400,  # Set the height of the graph in pixels
)

# Show the plot
fig.show()

C:\Users\20212189\AppData\Local\Temp\ipykernel_14044\3826111560.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\20212189\AppData\Local\Temp\ipykernel_14044\3826111560.py:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\20212189\AppData\Local\Temp\ipykernel_14044\3826111560.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.

In [26]:
# Drop rows where class is 'background'
df_non_background = dfv8[dfv8['Class'] != 'background']

# Group by threshold
thresholds = df_non_background['Threshold'].unique()

results = {}

for threshold in thresholds:
    subset = df_non_background[df_non_background['Threshold'] == threshold]

    # Compute totals for TP, FP, and FN
    total_tp = subset['TP'].sum()
    total_fp = subset['FP'].sum()
    total_fn = subset['FN'].sum()

    # Compute precision, recall, and F1 score
    precision = total_tp / (total_tp + total_fp) if (total_tp + total_fp) > 0 else 0
    recall = total_tp / (total_tp + total_fn) if (total_tp + total_fn) > 0 else 0
    f1_score = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    # Store results in a dictionary with the threshold as the key
    results[threshold] = [total_tp, total_fp, total_fn, precision, recall, f1_score]

print(results)

{0.001: [102859, 6109, 6054, 0.9439376697746127, 0.9444143490676045, 0.9441759492567043], 0.1: [103811, 14307, 5102, 0.8788753619262094, 0.953155270720667, 0.9145094722747115], 0.2: [103196, 7815, 5717, 0.9296015710154849, 0.9475085618796654, 0.9384696531529073], 0.3: [102571, 5215, 6342, 0.9516170931289778, 0.9417700366347451, 0.9466679587815355], 0.4: [100719, 3642, 8194, 0.965101905884382, 0.9247656386289974, 0.9445033149844799], 0.5: [97207, 2503, 11706, 0.9748972018854679, 0.8925197175727415, 0.9318914980610957], 0.6: [91264, 1757, 17649, 0.9811117919609551, 0.8379532287238438, 0.9038992938286767], 0.7: [79231, 1129, 29682, 0.9859507217521155, 0.7274705498884431, 0.8372139713535476], 0.8: [35806, 312, 73107, 0.9913616479317792, 0.3287578158713836, 0.4937702973846971], 0.9: [9, 0, 108905, 1.0, 8.263400481113539e-05, 0.00016525435399318783]}


In [35]:
def calculate_metrics(precision, recall, total_instances):
    # Start by assuming TP as an unknown and solve for TP, FP, FN
    # Precision = TP / (TP + FP)
    # Recall = TP / (TP + FN)

    # Let's assume TP as a variable and express FP and FN in terms of TP
    TP = total_instances * recall / (1 / precision + 1 / recall - 1)
    FN = total_instances - TP  # Since TP + FN = total_instances
    FP = TP * (1 / precision - 1)  # From Precision formula: FP = TP * (1 / precision - 1)

    # Now calculate F1 Score:
    f1_score = 2 * (precision * recall) / (precision + recall)

    return TP, FP, FN, f1_score

# Example usage
precision = 1  # Example precision value
recall = 0.000157     # Example recall value
total_instances = 108913  # Given total instances

TP, FP, FN, f1_score = calculate_metrics(precision, recall, total_instances)

print(f"True Positives (TP): {TP}")
print(f"False Positives (FP): {FP}")
print(f"False Negatives (FN): {FN}")
print(f"F1 Score: {f1_score}")

True Positives (TP): 0.002684596537
False Positives (FP): 0.0
False Negatives (FN): 108912.99731540347
F1 Score: 0.00031395070973857104


In [36]:
data = {'8': {0.001: [102859, 6109, 6054, 0.9439376697746127, 0.9444143490676045, 0.9441759492567043], 0.1: [103811, 14307, 5102, 0.8788753619262094, 0.953155270720667, 0.9145094722747115], 0.2: [103196, 7815, 5717, 0.9296015710154849, 0.9475085618796654, 0.9384696531529073], 0.3: [102571, 5215, 6342, 0.9516170931289778, 0.9417700366347451, 0.9466679587815355], 0.4: [100719, 3642, 8194, 0.965101905884382, 0.9247656386289974, 0.9445033149844799], 0.5: [97207, 2503, 11706, 0.9748972018854679, 0.8925197175727415, 0.9318914980610957], 0.6: [91264, 1757, 17649, 0.9811117919609551, 0.8379532287238438, 0.9038992938286767], 0.7: [79231, 1129, 29682, 0.9859507217521155, 0.7274705498884431, 0.8372139713535476], 0.8: [35806, 312, 73107, 0.9913616479317792, 0.3287578158713836, 0.4937702973846971], 0.9: [9, 0, 108905, 1.0, 8.263400481113539e-05, 0.00016525435399318783]},'9':{0.001: [103126, 6017, 5787, 0.9448704910072108, 0.9468658470522343, 0.9458671167039658], 0.1: [103974, 11991, 4939, 0.8965981114991592, 0.9546518781045421, 0.924714734211439], 0.2: [103449, 7501, 5464, 0.932392969806219, 0.9498315168988092, 0.9410314605004025], 0.3: [102790, 4933, 6123, 0.9542066225411472, 0.9437808158805652, 0.9489650842888531], 0.4: [100981, 3512, 7932, 0.9663900931162853, 0.927171228411668, 0.946374516180426], 0.5: [97761, 2444, 11152, 0.9756099995010229, 0.8976063463498388, 0.9349840759762431], 0.6: [92857, 1578, 16056, 0.9832900937152539, 0.8525795818680966, 0.9132816649290871], 0.7: [83243, 1000, 25670, 0.9881295775316643, 0.7643072911406352, 0.8619250761042888], 0.8: [44714, 328, 64199, 0.9927179077305626, 0.41054786848218305, 0.5808710337436264], 0.9: [3, 0, 108913, 1.0, 2.7544162473833045e-05, 5.508680762768663e-05]}  ,'10': {0.001: [103400, 11414, 5513, 0.9005870364241295, 0.9493816165196074, 0.9243408260960904], 0.1: [104615, 25573, 4298, 0.8035686852858942, 0.9605373095957324, 0.8750695312859418], 0.2: [103844, 18517, 5069, 0.8486691020831801, 0.9534582648535987, 0.8980170706607749], 0.3: [102581, 7854, 6332, 0.9288812423597591, 0.9418618530386639, 0.9353265131207031], 0.4: [99519, 3817, 9394, 0.963062243555005, 0.9137476701587506, 0.9377570683489675], 0.5: [96090, 2386, 12823, 0.975770746171656, 0.88226382525502, 0.9266643843212512], 0.6: [92209, 1657, 16704, 0.9823471757611915, 0.8466298788941632, 0.9094531485015707], 0.7: [86970, 1131, 21939, 0.9871624612660469, 0.7985565931190259, 0.882899345210903], 0.8: [75169, 594, 33745, 0.9921597613610865, 0.6901683897386929, 0.8140591410949928], 0.9: [4991, 3, 103923, 0.999399279134962, 0.045825146445819635, 0.0876321241703831]} ,'11':{0.001: [103003, 6944, 5910, 0.9368422967429761, 0.9457365052840341, 0.9412683907520789], 0.1: [103948, 16483, 4965, 0.8631332464232632, 0.9544131554543535, 0.9064810939026091], 0.2: [103325, 8840, 5588, 0.9211875362189631, 0.948692993490217, 0.9347379657858312], 0.3: [102619, 5646, 6294, 0.947850182422759, 0.942210755373555, 0.945022055640995], 0.4: [100398, 3643, 8515, 0.9649849578531541, 0.9218183320632064, 0.9429078580350686], 0.5: [96396, 2244, 15722, 0.9772506082725061, 0.8597727394352379, 0.9147553117793867], 0.6: [89868, 1346, 19045, 0.9852434933233933, 0.82513565873679, 0.898109700340284], 0.7: [76638, 661, 32275, 0.9914487897644213, 0.7036625563523179, 0.8231263291302388], 0.8: [27602, 58, 81311, 0.9979031091829357, 0.2534316380964623, 0.4042087381839749], 0.9: [8, 0, 108906, 1.0, 7.345244872100923e-05, 0.0001468941077101045]}}

In [37]:
# Extract the confidence scores and F1 scores for each model
confidence_scores = list(data['8'].keys())

# Create the figure
fig = go.Figure()
i = -1
colors = ['#2ca02c', '#ff7f0e', '#1f77b4', '#d62728']
# Add traces for each model
for model_key in ['8', '9', '10', '11']:
    i+=1
    f1_scores = [data[model_key][conf][5] for conf in confidence_scores]
    fig.add_trace(go.Scatter(
        x=confidence_scores,
        y=f1_scores,
        mode='lines+markers',
        name=f'YOLOV{model_key}',
        line=dict(width=3, color=colors[i]),
        marker=dict(size=8)

    ))

# Update layout
fig.update_layout(
    title="F1 score at various confidence thresholds",
    title_x=0.5,  # Center the title
    template="plotly",  # Use the plotly template
    xaxis=dict(
        title='Confidence threshold',
        tickvals=[0.001, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
        dtick=0.1,  # You can adjust this to control the interval
    ),
    yaxis=dict(
        title='F1-Score',
        range=[0, 1],
    ),
    showlegend=True,
    width=1000,  # Set the width of the graph in pixels
    height=400  # Set the height of the graph in pixels
)

# Add interactivity options for colors and intervals
fig.update_layout(
    dragmode='zoom',  # Allow zooming
    hovermode="x unified",  # Show hover info for all lines
)

fig.show()

In [38]:
# Extract the confidence scores and Precision scores for each model
confidence_scores = list(data['8'].keys())

# Create the figure
fig = go.Figure()

# List of colors for the models
i = -1
colors = ['#2ca02c', '#ff7f0e', '#1f77b4', '#d62728']

# Add traces for each model
for model_key in ['8', '9', '10', '11']:
    i += 1
    precision_scores = [data[model_key][conf][3] for conf in confidence_scores]  # Position 3 is Precision
    fig.add_trace(go.Scatter(
        x=confidence_scores,
        y=precision_scores,
        mode='lines+markers',
        name=f'YOLOV{model_key}',
        line=dict(width=3, color=colors[i]),
        marker=dict(size=8)
    ))

# Update layout
fig.update_layout(
    title="Precision at various confidence thresholds",
    title_x=0.5,  # Center the title
    template="plotly",  # Use the plotly template
    xaxis=dict(
        title='Confidence threshold',
        tickvals=[0.001, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
        dtick=0.1,  # You can adjust this to control the interval
    ),
    yaxis=dict(
        title='Precision',
        range=[0, 1.1],
    ),
    showlegend=True,
    width=1000,  # Set the width of the graph in pixels
    height=400  # Set the height of the graph in pixels
)

# Add interactivity options for colors and intervals
fig.update_layout(
    dragmode='zoom',  # Allow zooming
    hovermode="x unified",  # Show hover info for all lines
)

fig.show()

In [39]:
# Extract the confidence scores and Recall scores for each model
confidence_scores = list(data['8'].keys())

# Create the figure
fig = go.Figure()

# List of colors for the models
i = -1
colors = ['#2ca02c', '#ff7f0e', '#1f77b4', '#d62728']

# Add traces for each model
for model_key in ['8', '9', '10', '11']:
    i += 1
    recall_scores = [data[model_key][conf][4] for conf in confidence_scores]  # Position 4 is Recall
    fig.add_trace(go.Scatter(
        x=confidence_scores,
        y=recall_scores,
        mode='lines+markers',
        name=f'YOLOV{model_key}',
        line=dict(width=3, color=colors[i]),
        marker=dict(size=8)
    ))

# Update layout
fig.update_layout(
    title="Recall at various confidence thresholds",
    title_x=0.5,  # Center the title
    template="plotly",  # Use the plotly template
    xaxis=dict(
        title='Confidence threshold',
        tickvals=[0.001, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
        dtick=0.1,  # You can adjust this to control the interval
    ),
    yaxis=dict(
        title='Recall',
        range=[0, 1],
    ),
    showlegend=True,
    width=1000,  # Set the width of the graph in pixels
    height=400  # Set the height of the graph in pixels
)

# Add interactivity options for colors and intervals
fig.update_layout(
    dragmode='zoom',  # Allow zooming
    hovermode="x unified",  # Show hover info for all lines
)

fig.show()

In [40]:
# Assuming dfv8, dfv9, dfv10, and df are already defined pandas dataframes
def compute_f1_scores(df):
    # Filter rows where 'Class' == 'player'
    df_player = df[df['Class'] == 'referee']

    # Compute precision, recall, and F1 score
    df_player['Precision'] = df_player['TP'] / (df_player['TP'] + df_player['FP'])
    df_player['Recall'] = df_player['TP'] / (df_player['TP'] + df_player['FN'])
    df_player['F1'] = 2 * (df_player['Precision'] * df_player['Recall']) / (df_player['Precision'] + df_player['Recall'])

    # Group by 'Threshold' and compute the mean F1 score for each threshold
    return df_player.groupby('Threshold')['F1'].mean()

# Compute F1 scores for each dataframe
f1_v8 = compute_f1_scores(dfv8)
f1_v9 = compute_f1_scores(dfv9)
f1_v10 = compute_f1_scores(dfv10)
f1 = compute_f1_scores(df)

# Create the line plot
fig = go.Figure()
colors = ['#2ca02c', '#ff7f0e', '#1f77b4', '#d62728']
# Add traces for each dataframe
fig.add_trace(go.Scatter(x=f1_v8.index, y=f1_v8, mode='lines+markers', name='YOLOV8', line=dict(color=colors[0])))
fig.add_trace(go.Scatter(x=f1_v9.index, y=f1_v9, mode='lines+markers', name='YOLOV9', line=dict(color=colors[1])))
fig.add_trace(go.Scatter(x=f1_v10.index, y=f1_v10, mode='lines+markers', name='YOLOV10', line=dict(color=colors[2])))
fig.add_trace(go.Scatter(x=f1.index, y=f1, mode='lines+markers', name='YOLO11', line=dict(color=colors[3])))

# Customize the layout
fig.update_layout(
    title={'text': 'F1-score for referee class at various confidence thresholds', 'x': 0.5, 'xanchor': 'center'},
    xaxis_title='Confidence Threshold',
    yaxis_title='F1-Score',
    template='plotly',
    width=800,  # Set the width of the graph in pixels
    height=400,  # Set the height of the graph in pixels
)

# Show the plot
fig.show()

C:\Users\20212189\AppData\Local\Temp\ipykernel_14044\4055035623.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\20212189\AppData\Local\Temp\ipykernel_14044\4055035623.py:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\20212189\AppData\Local\Temp\ipykernel_14044\4055035623.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.

In [41]:
# Assuming dfv8, dfv9, dfv10, and df are already defined pandas dataframes
def compute_f1_scores(df):
    # Filter rows where 'Class' == 'player'
    df_player = df[df['Class'] == 'goalkeepers']

    # Compute precision, recall, and F1 score
    df_player['Precision'] = df_player['TP'] / (df_player['TP'] + df_player['FP'])
    df_player['Recall'] = df_player['TP'] / (df_player['TP'] + df_player['FN'])
    df_player['F1'] = 2 * (df_player['Precision'] * df_player['Recall']) / (df_player['Precision'] + df_player['Recall'])

    # Group by 'Threshold' and compute the mean F1 score for each threshold
    return df_player.groupby('Threshold')['F1'].mean()

# Compute F1 scores for each dataframe
f1_v8 = compute_f1_scores(dfv8)
f1_v9 = compute_f1_scores(dfv9)
f1_v10 = compute_f1_scores(dfv10)
f1 = compute_f1_scores(df)

# Create the line plot
fig = go.Figure()
colors = ['#2ca02c', '#ff7f0e', '#1f77b4', '#d62728']
# Add traces for each dataframe
fig.add_trace(go.Scatter(x=f1_v8.index, y=f1_v8, mode='lines+markers', name='YOLOV8', line=dict(color=colors[0])))
fig.add_trace(go.Scatter(x=f1_v9.index, y=f1_v9, mode='lines+markers', name='YOLOV9', line=dict(color=colors[1])))
fig.add_trace(go.Scatter(x=f1_v10.index, y=f1_v10, mode='lines+markers', name='YOLOV10', line=dict(color=colors[2])))
fig.add_trace(go.Scatter(x=f1.index, y=f1, mode='lines+markers', name='YOLO11', line=dict(color=colors[3])))

# Customize the layout
fig.update_layout(
    title={'text': 'F1-score of goalkeeper class at various confidence thresholds', 'x': 0.5, 'xanchor': 'center'},
    xaxis_title='Confidence Threshold',
    yaxis_title='F1-Score',
    template='plotly',
    width=800,  # Set the width of the graph in pixels
    height=400,  # Set the height of the graph in pixels
)

# Show the plot
fig.show()

C:\Users\20212189\AppData\Local\Temp\ipykernel_14044\956329545.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\20212189\AppData\Local\Temp\ipykernel_14044\956329545.py:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\20212189\AppData\Local\Temp\ipykernel_14044\956329545.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org